In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("table.csv")

In [3]:
df['SAMPLE_ID1'] = [item.rstrip('"').lstrip('"') for item in df.SAMPLE_ID1]

# delimiter patterns

In [4]:
delims = ["_","-"," "]

In [5]:
def delimpattern(s,delims):
    d = ''
    for i in s:
        for delim in delims:
            if i==delim:
                d += delim
                
    return d

In [6]:
df['spattern'] = [delimpattern(item,delims) for item in df.SAMPLE_ID1]

In [7]:
df.spattern.value_counts()

__-      65806
_-       19965
___        612
-           45
_-          42
__ -        12
--           9
Name: spattern, dtype: int64

There should only be two delimiter patterns (binder and sample book), but there are 7. Now let's deal with each individually.

In [8]:
dpatterns = list(df.spattern.value_counts().index)

In [9]:
def elementcheck(d):
    mf = d['mf']
    mloc = d['mloc']
    mtrial = d['mtrial']
    mmode = d['mmode']
    
    fails = 0
    
    if mf is not None:
        if len(mf) > 3:
            fails+=1
        try:
            int(mf[0])
            fails+=1
        except:
            pass
    
    if mloc is not None:
        if mloc not in ['dmax','dmin','dmix','din','dmim']:
            fails+=1
            
    try:
        int(mtrial)
    except:
        fail+=1
        
    if mmode is not None:
        if mmode not in ['M0','M1','M2']:
            fails+=1
            
    if fails > 0:
        return False
    else:
        return d

In [10]:
def mlocfixer(mloc):
    if mloc in ['dmax','dmix']:
        mloc = 'dmax'
    elif mloc in ['dmin','din','dmim']:
        mloc = 'dmin'
    else:
        pass
    
    return mloc

In [11]:
def dpattern0(s):
    cat = s.split('_')[0]
    mf = s.split('_')[1]
    mloc = mlocfixer(s.split('_')[2].split('-')[0])
    mtrial = s.split('-')[1].split(' ')[0]
    mmode = s.split(' ')[1]
    
    return {"cat":cat,"mf":mf,"mloc":mloc,"mtrial":mtrial,"mmode":mmode}

In [12]:
def dpattern1(s):
    cat = s.split('_')[0]
    mf = s.split('_')[1].split('-')[0]
    mloc = None
    mtrial = s.split('-')[1].split(' ')[0]
    mmode = s.split(' ')[1]
    
    return {"cat":cat,"mf":mf,"mloc":mloc,"mtrial":mtrial,"mmode":mmode}

In [13]:
def dpattern2(s):
    cat = s.split('_')[0]
    mf = s.split('_')[1]
    mloc = mlocfixer(s.split('_')[2])
    mtrial = s.split('_')[3].split(' ')[0]
    mmode = s.split(' ')[1]
    
    return {"cat":cat,"mf":mf,"mloc":mloc,"mtrial":mtrial,"mmode":mmode}

In [14]:
def dpattern3(s):
    cat = s.split('-')[0]
    mf = None
    mloc = None
    mtrial = s.split('-')[1].split(' ')[0]
    mmode = s.split(' ')[1]
    
    return {"cat":cat,"mf":mf,"mloc":mloc,"mtrial":mtrial,"mmode":mmode}

In [15]:
def dpattern4(s):
    cat = s.split('_')[0]
    mf = s.split('_')[1].split('-')[0]
    mloc = None
    mtrial = s.split('-')[1]
    mmode = None
    
    return {"cat":cat,"mf":mf,"mloc":mloc,"mtrial":mtrial,"mmode":mmode}

In [16]:
def dpattern5(s):
    cat = s.split('_')[0]
    mf = s.split('_')[1]
    mloc = mlocfixer(s.split('_')[2].split(' ')[0])
    mtrial = s.split('-')[1].split(' ')[0]
    mmode = s.split(' ')[2]
    
    return {"cat":cat,"mf":mf,"mloc":mloc,"mtrial":mtrial,"mmode":mmode}

In [17]:
def dpattern6(s):
    cat = s.split('-')[0]
    mf = s.split('-')[1]
    mloc = None
    mtrial = s.split('-')[2].split(' ')[0]
    mmode = s.split(' ')[1]
    
    return {"cat":cat,"mf":mf,"mloc":mloc,"mtrial":mtrial,"mmode":mmode}

In [18]:
pdict = {
    dpatterns[0]:dpattern0,
    dpatterns[1]:dpattern1,
    dpatterns[2]:dpattern2,
    dpatterns[3]:dpattern3,
    dpatterns[4]:dpattern4,
    dpatterns[5]:dpattern5,
    dpatterns[6]:dpattern6,
}

In [19]:
df['idxparse'] = [elementcheck(pdict[df.spattern.loc[i]](df.SAMPLE_ID1.loc[i])) for i in df.index]

We are eliminating the below IDs because they are not unique in the binders. They probably appear in the flag files, but just in case they don't...

In [20]:
badsampleids = list(set([item.split('_')[0] for item in df.SAMPLE_ID1[df.idxparse==False]]))

In [21]:
df = df[df.idxparse!=False]

In [22]:
parsedcols = pd.DataFrame(list(df.idxparse))

In [23]:
df = df.reset_index(drop=True)

In [24]:
df = pd.concat([parsedcols,df],axis=1)

In [25]:
del df['SAMPLE_ID2']
del df['SAMPLE_ID3']
del df['idxparse']
del df['spattern']

In [26]:
df = df[~df.cat.isin(badsampleids)]

# select mode

In [27]:
df = df[df.mmode=='M0']

In [28]:
df = df.reset_index(drop=True)

# repeat catalog numbers

Repeats are expected, of course, since there are generally 3 trials per paper. Sometimes, there are more, for whatever reason. However, what we need to find are cases where there are non-sequential repeats. If there is an error in entering a catalog number, it should produce nonsequential repeats. There is an outside chance that the operator could enter a correct catalog number, and then on the very next sample, enter that same number again, erroneously, but this seems so improbable as to be dismissed. 

In [29]:
import numpy as np

In [30]:
badcats = []
for cat in df.cat.unique():
    tmp = df[df.cat==cat]
    s = sorted(list(tmp.index))
    expected_list = list(np.arange(s[0],s[-1]+1,1))
    if s != expected_list:
        badcats.append(cat)

In [31]:
badcats_checked = []
for badcat in badcats:
    tmp = df[df.cat==badcat]
    try:
        fullidxs = [item.split('_')[0]+'_'+item.split('_')[1][:2] for item in tmp.SAMPLE_ID1]
        if len(list(set(fullidxs)))!=1:
            badcats_checked.append(badcat)
    except Exception as e:
        print(e)
        badcats_checked.append(badcat)

In [32]:
i = 0

In [33]:
print(i+1,'of',len(badcats_checked))
print(df.SAMPLE_ID1[df.cat==badcats_checked[i]])
i+=1

1 of 11
7914     2569_ps-1 M0
7915     2569_ps-2 M0
7916     2569_ps-3 M0
17068    2569_kp-1 M0
17069    2569_kp-2 M0
17070    2569_kp-3 M0
Name: SAMPLE_ID1, dtype: object


In [34]:
badidxs = ["2569_kp-","2240_kp-","2173_kk-","4945_kk-","2372nn_kt_","2372nn_kt_",
           "5154_ll-","2355_km-","369_dd-","2883_bp-","2092_kp-"]

In [35]:
todrop = []
for i in df.index:
    sampleid = df.SAMPLE_ID1.loc[i]
    for badidx in badidxs:
        if badidx in sampleid:
            todrop.append(i)

In [36]:
df = df.drop(todrop)

# flags

Dropping measurements can be important for color, so we'll be as liberal as possible in dropping here.

In [37]:
flagfiles = [
    '/Users/damoncrockett/Dropbox/lml/genome_2021/_flags/IgnoreList.xlsx',
    '/Users/damoncrockett/Dropbox/lml/genome_2021/_flags/20220125 Collection Measurement Issues and Notes.xlsx',
    '/Users/damoncrockett/Dropbox/lml/genome_2021/_flags/binder_flags_Crockett.xlsx',
    '/Users/damoncrockett/Dropbox/lml/genome_2021/_flags/ColorMeasuerementIssues_SampleBooks.xlsx'
]

In [38]:
ff = pd.read_excel(flagfiles[0])

In [39]:
df = df[~df.cat.isin(ff.catalog[ff.dim=='a'])]

In [40]:
ff = pd.read_excel(flagfiles[1])

In [41]:
ff = ff.fillna('')

In [42]:
df[df.cat.isin(ff['Sample:'][ff['Binder/Box:'].str.contains('Binder')])]

,cat,mf,mloc,mtrial,mmode,SAMPLE_ID1,LAB_L,LAB_A,LAB_B,nm380,...,nm670,nm680,nm690,nm700,nm710,nm720,nm730,LUV_L,LUV_U,LUV_V


In [43]:
ff = pd.read_excel(flagfiles[2])

In [44]:
df = df[~df.cat.isin(ff.catalog)]

In [45]:
ff = pd.read_excel(flagfiles[3])

In [46]:
ff = ff.fillna('')

In [47]:
todrop = list(df.index[(df.cat.isin(ff['Cat #'][ff.Issue.str.contains('min')]))&(df.mloc=='dmin')])

In [48]:
df = df.drop(todrop)

In [49]:
todrop = list(df.index[(df.cat.isin(ff['Cat #'][ff.Issue.str.contains('max')]))&(df.mloc=='dmax')])

In [50]:
df = df.drop(todrop)

In [51]:
df = df.reset_index(drop=True)

# LUV issue

In [52]:
from colormath.color_objects import LabColor,LuvColor
from colormath.color_conversions import convert_color

In [53]:
def luv2lab(i):
    l = df.LUV_L.loc[i]
    u = df.LUV_U.loc[i]
    v = df.LUV_V.loc[i]
    luv = LuvColor(l,u,v,observer='2',illuminant='d65')
    lab = convert_color(luv,LabColor)
    return lab.get_value_tuple()

In [54]:
luvidxs = df.index[df.LUV_L.notnull()]

In [55]:
for luvidx in luvidxs:
    lab = luv2lab(luvidx)
    df.LAB_L.loc[luvidx] = lab[0]
    df.LAB_A.loc[luvidx] = lab[1]
    df.LAB_B.loc[luvidx] = lab[2]

In [56]:
del df['LUV_L']
del df['LUV_U']
del df['LUV_V']

# save

In [57]:
df.to_csv('clean.csv',index=False)